In [1]:
import numpy as np
import pandas as pd
import glob
import sys
import h5py
#from netCDF4 import Dataset
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

import pyarrow as pa
import pyarrow.parquet as pqw

from functools import reduce
import operator
import gc

In [2]:
# plot settings
plt.rc('font', family='serif') 
plt.rc('font', serif='Times New Roman') 
plt.rcParams.update({'font.size': 16})
plt.rcParams['mathtext.fontset'] = 'stix'

# Initiate a spark session

In [3]:
# PySpark packages
from pyspark import SparkContext   
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import Row
from pyspark.sql.window import Window as W

#spark = SparkSession.builder \
#    .master("yarn") \
#    .appName("spark-shell") \
#    .config("spark.driver.maxResultSize", "32g") \
#    .config("spark.driver.memory", "32g") \
#    .config("spark.executor.memory", "6g") \
#    .config("spark.executor.cores", "1") \
#    .config("spark.executor.instances", "30") \
#    .getOrCreate()

#    .config("spark.executor.memory", "14g") \
#    .config("spark.executor.cores", "2") \
#    .config("spark.executor.instances", "60") \
#    .config("spark.executor.memory", "6g") \
#    .config("spark.jars.packages", "graphframes:graphframes:0.7.0-spark2.4-s_2.11") \
#    .getOrCreate()

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("spark://sohnic:7077") \
    .config("spark.driver.memory", "100g") \
    .getOrCreate()

sc = spark.sparkContext
sc.setCheckpointDir("hdfs://sohnic:54310/tmp/checkpoints")

spark.conf.set("spark.sql.debug.maxToStringFields", 500)
spark.conf.set("spark.sql.hive.filesourcePartitionFileCacheSize", 536870912)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/11 16:14:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# check the spark configuration
sc.getConf().getAll()[:10]

[('spark.rdd.compress', 'True'),
 ('spark.app.id', 'app-20251111161408-0145'),
 ('spark.master', 'spark://sohnic:7077'),
 ('spark.app.startTime', '1762845248072'),
 ('spark.sql.warehouse.dir',
  'file:/home/nowan/subhalo_kinematic/spark-warehouse'),
 ('spark.driver.port', '35169'),
 ('spark.app.name', 'MyApp'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.pyFiles', ''),
 ('spark.driver.host', 'sohnic')]

# Define particle data directories to be read

In [23]:
ptl_files = np.sort(glob.glob('/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted*.csv')) # pathes of particle data files

print(f"The number of particle data files: {len(ptl_files)}")
print("First five data files:")
print(ptl_files)

The number of particle data files: 1000
First five data files:
['/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y0_z0.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y0_z1.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y0_z2.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y0_z3.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y0_z4.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y0_z5.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y0_z6.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y0_z7.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y0_z8.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y0_z9.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y1_z0.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y1_z1.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap099_star_sorted_x0_y1_z2.csv'
 '/mnt/data1/TNG300/snap_099sorted/snap0

In [24]:
%%time
# An example data from a particle data file
df = pd.read_csv(ptl_files[0])
df.head()
#h5f.keys()

CPU times: user 200 ms, sys: 20.8 ms, total: 221 ms
Wall time: 237 ms


,px,py,pz,vx,vy,vz,mass,ID,Formation_time
0,1315.331632,4634.504183,1751.507651,-83.410126,-264.06277,-264.50668,0.000644,119207515343,0.208367
1,1315.021080,4634.467076,1751.345900,-90.334200,-233.31252,-305.05290,0.000460,120708871516,0.233722
2,1314.793927,4634.235272,1751.621171,-47.516525,-235.21378,-282.60388,0.000519,123002943475,0.273948
3,1315.273938,4634.281666,1751.768356,-19.892979,-259.25626,-280.80990,0.000490,122923618487,0.274130
4,1315.117454,4634.261051,1751.265073,-49.234486,-209.98532,-284.97310,0.000622,119207146242,0.207735


# Save selected features as a parquet
We will convert all particle data files from .csv to .parquet formats.

The blow cells show how to convert one particle data file into .parquet format.

If you need to convert all particle data files and do not need to know how to convert them individually, skip this section and go to the next section

## Construct a Spark DataFrame
First, we save the particle data from the particle data files into a Spark Data Frame named sparkdf

In [9]:
# Define a data structure for a Spark Data Frame
# T.StructType([...]): define a structured schema for a DataFrame
# T.StructField(name, data_type, nullable)
#  name: column name
#  data_type: Data type (IntegerType(), StringType(), ShortType(), DoubleType(), BooleanType(), ...)
#  nullable: If True, the column can have Null values
schema = T.StructType([\
                       T.StructField('px',T.FloatType(), True),\
                       T.StructField('py',T.FloatType(), True),\
                       T.StructField('pz',T.FloatType(), True),\
                       T.StructField('vx',T.FloatType(), True),\
                       T.StructField('vy',T.FloatType(), True),\
                       T.StructField('vz',T.FloatType(), True),\
                       T.StructField('mass',T.FloatType(), True),\
                      ])

In [21]:
%%time
# Generate a Spark Data Frame named spakrdf according to the data in the pandas dataframe and the data type of schema
sparkdf = spark.createDataFrame(df[['px', 'py', 'pz', 'vx', 'vy', 'vz', 'mass']],schema)

/usr/local/spark/python/pyspark/sql/pandas/serializers.py:229: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(s.dtype):


CPU times: user 47.1 ms, sys: 11.1 ms, total: 58.1 ms
Wall time: 275 ms


In [18]:
%%time
print("Top three rows of sparkdf:")
sparkdf.show(3,truncate=True)
print()

nrow = sparkdf.count()
print(f"The number of rows: {nrow}")
print()

print("The structure of sparkdf:")
print(sparkdf.printSchema())
print()

Top three rows of sparkdf:
+---------+--------+---------+---------+--------+---------+-----------+
|       px|      py|       pz|       vx|      vy|       vz|       mass|
+---------+--------+---------+---------+--------+---------+-----------+
|4816.9507|943.8858|11228.532| 306.1135|78.89162|114.42368|7.473642E-5|
| 4817.317|945.4705| 11228.47|296.45004|84.64811| 94.15483|8.121476E-5|
| 4817.385|  943.36|11228.005|276.48822|97.95877|118.08153|5.140894E-5|
+---------+--------+---------+---------+--------+---------+-----------+
only showing top 3 rows

The number of rows: 25271
The structure of sparkdf:
root
 |-- px: float (nullable = true)
 |-- py: float (nullable = true)
 |-- pz: float (nullable = true)
 |-- vx: float (nullable = true)
 |-- vy: float (nullable = true)
 |-- vz: float (nullable = true)
 |-- mass: float (nullable = true)

None
CPU times: user 5.78 ms, sys: 6.49 ms, total: 12.3 ms
Wall time: 479 ms


# Converting all particle files (here, .csv file) into parquet files

Because the summation of all particles are so large, I save them in a batched parquet files

In [25]:
%%time

schema = T.StructType([\
                       T.StructField('px',T.FloatType(), True),\
                       T.StructField('py',T.FloatType(), True),\
                       T.StructField('pz',T.FloatType(), True),\
                       T.StructField('vx',T.FloatType(), True),\
                       T.StructField('vy',T.FloatType(), True),\
                       T.StructField('vz',T.FloatType(), True),\
                       T.StructField('mass',T.FloatType(), True),\
                        T.StructField('id', T.IntegerType(), True),\
                        T.StructField('formation_time', T.FloatType(), True),\
                      ])

n_batches = 10 # recommend to set it same as the number you use to divide the data cube for sorting particles
batch_size = len(ptl_files) // n_batches

for batch_idx in range(n_batches):
    print(f"Processing batch {batch_idx + 1} / {n_batches} ...")
    batch_files = ptl_files[batch_idx * batch_size : min((batch_idx + 1) * batch_size, len(ptl_files))]
    # create an empty spark DataFrame for saving all particle data
    sparkdf = spark.createDataFrame([], schema)

    for file in tqdm(batch_files, desc=f"Batch {batch_idx}", leave=False):
        # read i-th data file (.csv) of the particle data files and convert to a spark DataFrame
        df = pd.read_csv(file)
        tempdf = spark.createDataFrame(df[['px', 'py', 'pz', 'vx', 'vy', 'vz', 'mass', 'ID', 'Formation_time']], schema)
        # Append to the spark DataFrame for all particle data
        sparkdf = sparkdf.union(tempdf)

    # Save the batch
    outfile = f"hdfs://sohnic:54310/data/TNG300/snap_099/star_particle_batch{batch_idx}.parquet"
    sparkdf.write.option("compression", "snappy").mode("overwrite").save(outfile)
    # .option(key, value): set saving configuration.
    # "compression": determines the compression codec for saving the spark DataFrame.
    # "snappy" is a default compression option for Parquet. It is very fast and has a moderate compression ratio.
    # other options are Gzip, Bzip2, LZ4, ZSTD. Ask ChatGPT if you want to know their properties.
    # mode: specifies what happens if the output file already exists.
    # "overwrite": overwrite an existing files at the output path.
    # "append": add new data to existing files.
    # "ignore": skips writing if the output file already exists.
    # "error" or "errorifexists": raise an error if the output file exists.
    print(f"✔️ Saved batch {batch_idx} to {outfile}")

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/spark/python/pyspark/sql/pandas/serializers.py:229: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(s.dtype):
/usr/local/spark/python/pyspark/sql/pandas/serializers.py:229: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(s.dtype):
/usr/local/spark/python/pyspark/sql/pandas/serializers.py:229: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(s.dtype):
/usr/local/spark/python/pyspark/sql/pandas/serializers.py:229: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(s.dtype):
/usr/local/spark/python/pyspark/sql/pand

KeyboardInterrupt: 

# Read the saved parquet file

In [16]:
%%time
newsparkdf = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(outfile)
# hearder=="true": use the first row to infer column names. Actually, it is unnecessary if you read a Parquet file because the Parquet file independently stores column names in its format.
# recursiveFileLookup="true"
#  Allows Spark to recursively search through subdirectories within the specified path (outname) for Parquet files.
#  If outname is a directory with nested folders, Spark will read all Parquet files within those folders.

25/11/04 15:04:57 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


CPU times: user 3.16 ms, sys: 2.92 ms, total: 6.09 ms
Wall time: 1.02 s


In [4]:
%%time
outfile = "hdfs://sohnic:54310/data/TNG300/snap_099/star_particle.parquet"
newsparkdf = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(outfile)
# hearder=="true": use the first row to infer column names. Actually, it is unnecessary if you read a Parquet file because the Parquet file independently stores column names in its format.
# recursiveFileLookup="true"
#  Allows Spark to recursively search through subdirectories within the specified path (outname) for Parquet files.
#  If outname is a directory with nested folders, Spark will read all Parquet files within those folders.

CPU times: user 2.36 ms, sys: 3.83 ms, total: 6.19 ms
Wall time: 2.44 s


In [5]:
%%time
# summarize the spark Data Frame
newsparkdf.describe().toPandas()
# describe(): summarize the data in the spark Data Frame
# .toPandas(): convert the spark Data Frame into a pandas data frame

CPU times: user 46.8 ms, sys: 16 ms, total: 62.8 ms
Wall time: 3.05 s


,summary,px,py,pz,vx,vy,vz,mass,id,formation_time
0,count,0,0,0,0,0,0,0,0,0
1,mean,None,None,None,None,None,None,None,None,None
2,stddev,None,None,None,None,None,None,None,None,None
3,min,None,None,None,None,None,None,None,None,None
4,max,None,None,None,None,None,None,None,None,None


# Saving subhalo catalog

In [5]:
#subhalo table (~few seconds)
t300subhalo = pd.read_csv('subhalocat300.txt', sep=' ')

h = 0.7
t300subhalo = t300subhalo[t300subhalo.StarHalfRad*2*h>1e-1]

t300subhalo.head()

,SubfindID,px,py,pz,vx,vy,vz,StarHalfMass,StarHalfRad
0,0,43718.812500,48813.640625,147594.953125,472.196198,450.850006,-260.746918,265.473969,130.065216
1,1,45442.273438,51850.199219,146416.500000,-209.056656,-735.888916,400.641724,126.831070,118.332764
2,2,44490.761719,49091.714844,147870.578125,2021.729492,1495.440186,-1797.082153,28.682310,13.269690
3,3,43820.785156,50939.398438,147711.046875,925.150391,-473.445465,-275.925934,11.954713,14.077825
4,4,44302.578125,49630.972656,147869.484375,-260.214630,-2221.625244,-563.641296,11.029386,11.148003


In [6]:
%%time

schema_sub = T.StructType([\
                       T.StructField('px',T.FloatType(), True),\
                       T.StructField('py',T.FloatType(), True),\
                       T.StructField('pz',T.FloatType(), True),\
                       T.StructField('vx',T.FloatType(), True),\
                       T.StructField('vy',T.FloatType(), True),\
                       T.StructField('vz',T.FloatType(), True),\
                       T.StructField('sub_id',T.IntegerType(), True),\
                        T.StructField('StarHalfRad', T.FloatType(), True)
                      ])

SubhaloDf = spark.createDataFrame(t300subhalo[['px', 'py', 'pz', 'vx', 'vy', 'vz','SubfindID', 'StarHalfRad']], schema_sub)
SubhaloFile = 'hdfs://sohnic:54310/data/TNG300/snap_099/subhalo.parquet.snappy'
SubhaloDf.write.option("compression", "snappy").mode("overwrite").save(SubhaloFile)
# SubhaloDf.show(4)

/usr/local/spark/python/pyspark/sql/pandas/serializers.py:229: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(s.dtype):
25/11/11 16:14:35 WARN TaskSetManager: Stage 0 contains a task of very large size (1502 KiB). The maximum recommended task size is 1000 KiB.


CPU times: user 225 ms, sys: 35.3 ms, total: 260 ms
Wall time: 9.66 s


# repartition

In [24]:
%%time
# similar to "refresh"
newsparkdf.cache()
newsparkdf.repartition(10,"px").count()

CPU times: user 4.09 ms, sys: 40 µs, total: 4.13 ms
Wall time: 743 ms


765683